# Canonical ensemble Monte Carlo of bulkd SPC/E water


In this example, we reproduce the average energy reported in https://doi.org/10.1063/1.476834

In [1]:
import sys
import unittest
import feasst
sys.path.insert(0, feasst.install_dir() + '/plugin/ewald/tutorial/')
import spce
sys.path.insert(0, feasst.install_dir() + '/plugin/monte_carlo/tutorial/')
import analyze

class TestEwald1SPCENVT(unittest.TestCase):
    """Test a canonical ensemble Lennard Jones Monte Carlo simulation"""
    def test_srsw(self, num_particles=512, box_length=24.8586887, steps_per=1e5, temperature=298):
        """Compare with the reported average energy from https://doi.org/10.1063/1.476834.

        num_particles -- number of water molecules
        box_length -- box length in angstroms
        steps_per -- steps between each Anaylze/Modify
        temperature -- in Kelvin
        """

        monte_carlo = feasst.MonteCarlo()
        # monte_carlo.set(feasst.MakeRandomMT19937(feasst.args({"seed": "1234"})))    
        config = feasst.Configuration(feasst.MakeDomain(feasst.args({"cubic_box_length": str(box_length)})),
            feasst.args({"particle_type": feasst.install_dir() + '/forcefield/data.spce',
                         "physical_constants": "CODATA2010"}))
        monte_carlo.set(spce.system(
            config,
            alphaL=5.6,
            kmax_squared=38,
            rcut=config.domain().min_side_length()/2.
        ))
        R = monte_carlo.configuration().physical_constants().ideal_gas_constant()
        monte_carlo.set(feasst.MakeMetropolis(feasst.args(
            {"beta": str(1./(R*temperature/1e3)),
             "chemical_potential": "1."})))
        monte_carlo.add(feasst.MakeTrialTranslate(feasst.args({
            "weight": "1.",
            "tunable_param": "0.275",
        })))
        monte_carlo.add(feasst.MakeTrialRotate(feasst.args({ 
            "weight": "1.",
            "tunable_param": "50",
        })))
        feasst.SeekNumParticles(num_particles).with_trial_add().run(monte_carlo)
        analyze.add(monte_carlo, steps_per, tolerance=1e-6)

        # equilibrate
        monte_carlo.attempt(int(1e6))

        # compute average energy using a stepper/analysis and output into file
        energy = feasst.MakeEnergy(feasst.args({
            "steps_per_update": "1",
            "steps_per_write": str(steps_per),
            "file_name": "spce_nvt_energy.txt",
        }))
        monte_carlo.add(energy)

        # compute average using this script
        energy_alt = feasst.Accumulator()

        # production
        for _ in range(int(1e6)):
            monte_carlo.attempt()
            energy_alt.accumulate(monte_carlo.criteria().current_energy())

        # test that the two methods to compute average energy are the same
        self.assertAlmostEqual(energy.energy().average(), energy_alt.average(), delta=1e-6)

        # test the average against the NIST SRSW
        num = monte_carlo.configuration().num_particles()
        stdev = (energy.energy().block_stdev()**2 + (0.02*num)**2)**(1./2.)
        print(energy.energy().average(), energy_alt.average(), stdev)
        self.assertAlmostEqual(-46.82*num, energy.energy().average(), delta=8*stdev)

If the test passes, the energy is within the tolerance of the SRSW value and the two ensemble average methods agreed.

In [2]:
%%time
unittest.main(argv=[''], verbosity=2, exit=False)

test_srsw (__main__.TestEwald1SPCENVT)
Compare with the reported average energy from https://doi.org/10.1063/1.476834. ... 

-23866.439677679402 -23866.439677679402 36.42517383194891
CPU times: user 25min 52s, sys: 641 ms, total: 25min 52s
Wall time: 25min 51s


ok

----------------------------------------------------------------------
Ran 1 test in 1551.506s

OK
